In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, Flatten, MaxPooling1D
from keras.preprocessing import text, sequence
#import wandb
#from wandb.keras import WandbCallback
import numpy as np
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
dataset1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/base-case/80_20_split.csv')

In [ ]:
model = Sequential()
model.add(Embedding(90000, 50, input_length = 1000))
model.add(Dropout(0.5))
model.add(Conv1D(100, 3, padding = 'valid', activation = 'relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100, 4, padding = 'valid', activation = 'relu'))
model.add(MaxPooling1D())
model.add(Conv1D(100, 5, padding = 'valid', activation = 'relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(250, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units =3,  activation='softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
def CNN():
    dataset = dataset1.sample(n = 10000)
    X = dataset[['OBS(mist-free)']]
    X = pd.DataFrame(X)
    lb_make = LabelEncoder()
    dataset['Category-encoded'] = lb_make.fit_transform(dataset['Category'])
    Y = dataset[['Category-encoded']]
    Y = pd.DataFrame(Y)

    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, stratify = Y)
    tokenizer = text.Tokenizer(num_words=90000)
    tokenizer.fit_on_texts(X_train['OBS(mist-free)'].to_list())
    X_train1 = tokenizer.texts_to_sequences(X_train['OBS(mist-free)'].to_list())
    X_test1 = tokenizer.texts_to_sequences(X_test['OBS(mist-free)'].to_list())
    X_train = sequence.pad_sequences(X_train1, maxlen=1000)
    X_test = sequence.pad_sequences(X_test1, maxlen=1000)
    model.fit(X_train, Y_train, batch_size=32, epochs=30)

    y_pred_train = model.predict(X_train)
    predictions = np.argmax(y_pred_train, axis=-1)
    b = (classification_report(Y_train, predictions)).split()
    c = [float(b[7]), float(b[12]), float(b[17]), float(b[20])]

    y_pred = model.predict(X_test)
    predictions = np.argmax(y_pred, axis=-1)
    d = (classification_report(Y_test, predictions)).split()
    e = [float(b[7]), float(b[12]), float(b[17]), float(b[20]), float(d[7]), float(d[12]), float(d[17]), float(d[20])]

    return e

In [ ]:
cnn = []
for i in range(100):
    result = CNN()
    cnn.append(result)
    #print(result)
f = pd.DataFrame(cnn)
f.to_csv("/content/drive/MyDrive/Colab Notebooks/epochs-variation/cnn(10000)_epoch_30(100,[3,4,5].csv")

Epoch 1/30
250/250 [==============================] - 20s 51ms/step - loss: 1.0218 - accuracy: 0.5246
Epoch 2/30
250/250 [==============================] - 13s 51ms/step - loss: 0.8360 - accuracy: 0.6236
Epoch 3/30
250/250 [==============================] - 13s 51ms/step - loss: 0.6619 - accuracy: 0.7159
Epoch 4/30
250/250 [==============================] - 12s 50ms/step - loss: 0.5685 - accuracy: 0.7827
Epoch 5/30
250/250 [==============================] - 13s 51ms/step - loss: 0.5051 - accuracy: 0.8080
Epoch 6/30
250/250 [==============================] - 13s 51ms/step - loss: 0.4611 - accuracy: 0.8302
Epoch 7/30
250/250 [==============================] - 13s 51ms/step - loss: 0.4089 - accuracy: 0.8508
Epoch 8/30
250/250 [==============================] - 13s 50ms/step - loss: 0.3832 - accuracy: 0.8619
Epoch 9/30
250/250 [==============================] - 13s 50ms/step - loss: 0.3487 - accuracy: 0.8734
Epoch 10/30
250/250 [==============================] - 13s 50ms/step - loss: 0.337

In [6]:
#train['stopwords'] = train['OBS(mist-free)'].apply(lambda x: [x for x in x.split() if x in stop])
freq = pd.Series(' '.join(dataset1['OBS(mist-free)']).split()).value_counts()

z = pd.DataFrame(freq)
z.to_csv('/content/drive/MyDrive/Colab Notebooks/freq.csv')